In [ ]:
# Mount Google Drive (for uploading policy + invoices from your drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

from google.colab import files

# Upload all 3 ZIPs manually
uploaded = files.upload()

Saving Cab Bills.zip to Cab Bills.zip
Saving Travel Bill.zip to Travel Bill.zip
Saving Meal Invoice.zip to Meal Invoice.zip


In [ ]:
import os, glob, pprint

print(" Top‑level files and folders in /content:")
pprint.pprint(os.listdir("/content"))

print("\n Any ZIP files anywhere under /content:")
for f in glob.glob("/content/**/*.zip", recursive=True):
    print(" •", f)


 Top‑level files and folders in /content:
['.config',
 'Cab Bills.zip',
 'drive',
 'Meal Invoice.zip',
 'Travel Bill.zip',
 'sample_data']

 Any ZIP files anywhere under /content:
 • /content/Cab Bills.zip
 • /content/Meal Invoice.zip
 • /content/Travel Bill.zip


In [ ]:
import zipfile, os

# Exact file paths from /content
meal_zip   = "/content/Meal Invoice.zip"
travel_zip = "/content/Travel Bill.zip"
cab_zip    = "/content/Cab Bills.zip"   # Note: filename is Cab **Bills**

# Create folders to extract into
os.makedirs("data/invoices/meal", exist_ok=True)
os.makedirs("data/invoices/travel", exist_ok=True)
os.makedirs("data/invoices/cab", exist_ok=True)

# Extract all
with zipfile.ZipFile(meal_zip, 'r') as zip_ref:
    zip_ref.extractall("data/invoices/meal")

with zipfile.ZipFile(travel_zip, 'r') as zip_ref:
    zip_ref.extractall("data/invoices/travel")

with zipfile.ZipFile(cab_zip, 'r') as zip_ref:
    zip_ref.extractall("data/invoices/cab")

print("All invoices extracted into 'meal', 'travel', and 'cab' folders.")

All invoices extracted into 'meal', 'travel', and 'cab' folders.


In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is connected:", torch.cuda.get_device_name(0))
else:
    print("GPU is NOT connected.")

GPU is connected: Tesla T4


In [ ]:
!pip install pdfplumber
import pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 100.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files

# Upload PDF manually from your computer
uploaded = files.upload()

Saving Policy-Nov-2024.pdf to Policy-Nov-2024.pdf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print("Uploaded files in /content:")
print(os.listdir("/content"))

Uploaded files in /content:
['.config', 'data', 'Cab Bills.zip', 'Policy-Nov-2024.pdf', 'drive', 'Meal Invoice.zip', 'Travel Bill.zip', 'sample_data']


In [ ]:
import shutil
import os

# Create folder if it doesn't exist
os.makedirs("data/policy", exist_ok=True)

# Move the uploaded file to data/policy/
shutil.move("/content/Policy-Nov-2024.pdf", "data/policy/Policy-Nov-2024.pdf")

print("Policy file moved to data/policy/ folder.")

Policy file moved to data/policy/ folder.


In [ ]:
# Set the correct policy path
policy_path = "data/policy/Policy-Nov-2024.pdf"

# Function to extract text from any PDF
def extract_text_from_pdf(path):
    import pdfplumber
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                text += page.extract_text() + "\n"
    return text.strip()

# Load the policy text
policy_text = extract_text_from_pdf(policy_path)
print("Policy loaded. Preview:\n")
print(policy_text[:800])

Policy loaded. Preview:

Company Name: IAI Solution
Policy Title: Employee Reimbursement Policy
Version: 1.0
1. Purpose
The purpose of this policy is to outline the guidelines and procedures for the reimbursement of
expenses incurred by employees while performing work-related duties. This policy ensures
transparency and consistency in the reimbursement process.
2. Scope
This policy applies to all employees of IAI Solution who incur expenses in the course of their
work duties.
3. Reimbursement Categories
The following categories of expenses are eligible for reimbursement under this policy:
● Food and Beverages
● Travel Expenses
● Accommodations
4. General Guidelines
● All reimbursements must be supported by original receipts and submitted within 30 days
of the expense incurred.
● Employees must complete the reimbur


In [ ]:
# Move all files from 'Cab Bills' to 'cab'
!mv data/invoices/cab/Cab\ Bills/* data/invoices/cab/

# Move all files from 'Travel Bill' to 'travel'
!mv data/invoices/travel/Travel\ Bill/* data/invoices/travel/

In [ ]:
!rmdir data/invoices/cab/Cab\ Bills
!rmdir data/invoices/travel/Travel\ Bill

In [ ]:
!pip install -q openai

In [ ]:
import openai

openai.api_key = "*"
openai.api_base = "https://api.groq.com/openai/v1"

In [ ]:
# Truncate helper
def truncate(text, max_words=120):
    return " ".join(text.split()[:max_words])

# Prompt builder
def generate_prompt(policy, invoice_text, category):
    return f"""
You are a reimbursement assistant.

Policy:
\"\"\"
{policy}
\"\"\"

Invoice ({category.upper()}):
\"\"\"
{invoice_text}
\"\"\"

Only reply in this format:

Status: <Fully Reimbursed / Partially Reimbursed / Declined>
Reason: <1 sentence reason>
"""

# Load & truncate policy
policy_text = extract_text_from_pdf("data/policy/Policy-Nov-2024.pdf")
policy_short = truncate(policy_text, max_words=100)

In [ ]:
policy_path = "data/policy/Policy-Nov-2024.pdf"

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key="*",
)

def call_groq(prompt, model="llama3-8b-8192"):
    chat = client.chat.completions.create(
        model=model,
        messages=[
            {"role":"system", "content":"You are a reimbursement assistant. Reply in Status/Reason format."},
            {"role":"user",   "content":prompt}
        ],
        temperature=0.2,
        max_tokens=150,
    )
    return chat.choices[0].message.content.strip()

In [ ]:
!sudo apt-get install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 0s (2,242 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previousl

In [ ]:
!pip install pdf2image pytesseract

In [ ]:
from pdf2image import convert_from_path
import pytesseract

def extract_text_from_pdf_ocr(pdf_path):
    images = convert_from_path(pdf_path)
    full_text = ""
    for img in images:
        text = pytesseract.image_to_string(img)
        full_text += text + "\n"
    return full_text.strip()

In [ ]:
import re

def rule_based_check(text: str, category: str):
    text_lower = text.lower()
    lines = [ln.strip() for ln in text_lower.splitlines() if ln.strip()]

    def extract_amount(text, keywords):
        max_val = 0
        lines = text.lower().splitlines()

        for ln in lines:
            if any(k in ln for k in keywords):
                # Fix common OCR mistakes (e.g., split numbers like "21 00")
                clean_line = ln.replace(",", "")
                matches = re.findall(r"(\d{1,3}(?:[\s,]\d{2,3})+|\d+\.\d+|\d+)", clean_line)

                for match in matches:
                    number = match.replace(" ", "")  # merge "21 00" → "2100"
                    try:
                        val = float(number)
                        if 10 <= val <= 50000:  # reasonable range for travel
                            max_val = max(max_val, val)
                    except:
                        continue
        return max_val



    # Helper: find first number on a line containing any of these keywords
    def find_amount(keywords, low=1, high=100000):
        for ln in lines:
            if any(k in ln for k in keywords):
                nums = re.findall(r"([\d]{1,6}(?:\.\d{1,2})?)", ln)
                nums = [float(n.replace(",", "")) for n in nums]
                for n in reversed(nums):          # prefer last number on the line
                    if low <= n <= high:
                        return n
        return 0.0

    # ------------ MEAL ------------
    if category == "meal":
        alcohol_kw = ["whisky","rum","vodka","wine","beer","scotch","gin","tequila","alcohol","stag","mc","royal"]
        food_kw    = ["biryani","biriyani","idli","dosa","chapati","meal","meals","tea","coffee","roti","vada",
                      "thali","mini","paratha","sandwich","biriyvani"]

        alcohol_found = any(any(a in ln for a in alcohol_kw) for ln in lines)
        food_found    = any(any(f in ln for f in food_kw)   for ln in lines)

        # amounts
        subtotal = find_amount(["sub total","subtotal"],      10, 10000)
        cgst     = find_amount(["cgst"],                      0,   500)
        sgst     = find_amount(["sgst"],                      0,   500)
        total    = find_amount(["total"],                     10, 10000)

        if total == 0 and subtotal:
            total = subtotal + cgst + sgst

        if not food_found and alcohol_found:
            return "Declined", "Only alcohol items found, not reimbursable."
        if not food_found and total == 0:
            return "Declined", "No valid food items found."

        if total <= 200 and not alcohol_found:
            return "Fully Reimbursed", f"Meal within ₹200 limit (≈ ₹{int(total)}), no alcohol."
        if total > 200 and not alcohol_found:
            return "Partially Reimbursed", f"Meal amount ≈ ₹{int(total)} exceeds ₹200 per‑meal limit."
        # alcohol + food
        if alcohol_found:
            reimb = min(total, 200)
            return "Partially Reimbursed", f"Alcohol excluded. ₹{int(reimb)} reimbursable."
        return "Declined", "Could not classify meal invoice."

    # ---- TRAVEL CATEGORY RULES ----
    elif category == "travel":
        travel_keywords = ["ticket", "air india", "train", "flight", "airasia", "indigo", "goair", "ixigo"]
        if not any(kw in text.lower() for kw in travel_keywords):
            return "Declined", "No valid travel reference found."

        # First look for the most accurate label
        total_fare = extract_amount(text, ["total fare"])
        if total_fare:
            amount = total_fare
        else:
            amount = extract_amount(text, ["total", "fare", "amount", "price", "cost", "rs", "inr"])

        if amount == 0:
            return "Declined", "No valid travel cost found."

        if amount <= 2000:
            return "Fully Reimbursed", f"Trip cost ≈ ₹{int(amount)} within ₹2000 limit."
        else:
            return "Partially Reimbursed", f"Trip cost ≈ ₹{int(amount)} exceeds ₹2000 per‑trip limit."




    #---- CAB FARES ----
    elif category == "cab":
        keywords = ["total", "fare", "amount", "ride", "subtotal", "amt", "charges", "bill"]
        max_fare = 0

        for line in text.lower().splitlines():
            line_clean = re.sub(r"[^\w\s.\d₹]", "", line)
            line_clean = re.sub(r"(?<=\d)\s(?=\d)", "", line_clean)  # merge digits only
            if any(k in line_clean for k in keywords):
                numbers = re.findall(r"\d{2,4}(?:\.\d{1,2})?", line_clean)
                for n in numbers:
                    fare = float(n)
                    if 30 <= fare <= 1000 and fare > max_fare:
                        max_fare = fare

        if max_fare == 0:
            # fallback: scan all lines
            for line in text.lower().splitlines():
                line_clean = re.sub(r"[^\w\s.\d₹]", "", line)
                line_clean = re.sub(r"(?<=\d)\s(?=\d)", "", line_clean)
                numbers = re.findall(r"\d{2,4}(?:\.\d{1,2})?", line_clean)
                for n in numbers:
                    fare = float(n)
                    if 30 <= fare <= 1000 and fare > max_fare:
                        max_fare = fare

        if max_fare == 0:
            return "Declined", "No valid cab fare found."
        elif max_fare <= 150:
            return "Fully Reimbursed", f"Cab fare ≈ ₹{int(max_fare)} within ₹150 daily limit."
        else:
            return "Partially Reimbursed", f"Cab fare ≈ ₹{int(max_fare)} exceeds ₹150 daily cab limit."



In [ ]:
import os, time, re

policy_path  = "data/policy/Policy-Nov-2024.pdf"
policy_text  = extract_text_from_pdf(policy_path)
policy_short = truncate(policy_text, max_words=100)

results = []

for category in ["meal","travel","cab"]:
    folder = f"data/invoices/{category}"
    for file in os.listdir(folder):
        if not file.lower().endswith(".pdf"):
            continue

        filepath       = os.path.join(folder, file)
        invoice_text   = extract_text_from_pdf_ocr(filepath)
        invoice_short  = truncate(invoice_text, max_words=120)

        # -------- RULE‑BASED CHECK FIRST --------
        status, reason = rule_based_check(invoice_text, category)

        # -------- If no rule matched, use Groq LLM --------
        if status is None:                           # only travel/cab for now
            prompt   = generate_prompt(policy_short, invoice_short, category)
            response = call_groq(prompt)
            time.sleep(1)

            try:
                status_line = [l for l in response.split('\n') if "Status:" in l][0]
                reason_line = [l for l in response.split('\n') if "Reason:" in l][0]
                status = status_line.replace("Status:", "").strip()
                reason = reason_line.replace("Reason:", "").strip()
            except IndexError:
                status = "UNKNOWN"
                reason = "Could not parse model output"

        # -------- Print & store --------
        print(f"\n{file} | {category.upper()}")
        print(f"Status: {status}\nReason: {reason}")

        results.append(
            {
                "filename": file,
                "category": category,
                "status":   status,
                "reason":   reason,
                "text":     invoice_short,
            }
        )

print("\n Invoice analysis complete.")


Meal Invoice 6.pdf | MEAL
Status: Partially Reimbursed
Reason: Alcohol excluded. ₹200 reimbursable.

Meal Invoice 5.pdf | MEAL
Status: Partially Reimbursed
Reason: Meal amount ≈ ₹400 exceeds ₹200 per‑meal limit.

Meal Invoice 2.pdf | MEAL
Status: Partially Reimbursed
Reason: Meal amount ≈ ₹515 exceeds ₹200 per‑meal limit.

Meal Invoice 7.pdf | MEAL
Status: Partially Reimbursed
Reason: Alcohol excluded. ₹90 reimbursable.

Meal Invoice 4.pdf | MEAL
Status: Fully Reimbursed
Reason: Meal within ₹200 limit (≈ ₹80), no alcohol.

Meal Invoice 8.pdf | MEAL
Status: Partially Reimbursed
Reason: Alcohol excluded. ₹200 reimbursable.

Meal Invoice 1.pdf | MEAL
Status: Partially Reimbursed
Reason: Alcohol excluded. ₹200 reimbursable.

Meal Invoice 3.pdf | MEAL
Status: Partially Reimbursed
Reason: Meal amount ≈ ₹340 exceeds ₹200 per‑meal limit.

Book 7.pdf | TRAVEL
Status: Partially Reimbursed
Reason: Trip cost ≈ ₹2100 exceeds ₹2000 per‑trip limit.

Book 9.pdf | TRAVEL
Status: Partially Reimbursed
R

In [ ]:
results

[{'filename': 'Meal Invoice 6.pdf',
  'category': 'meal',
  'status': 'Partially Reimbursed',
  'reason': 'Alcohol excluded. ₹200 reimbursable.',
  'text': 'Slogo ra WELCOME Manish Resort 12/2 east Bengalore Original Receipt Date: 26 Dec 2024 Time: 20:38 Avinash Table: #001 Receipt No.: 0001 Description Qty Price Subtotal Wines 5 x 150 ¢ 750 Biriyvani 2 3149 < 298 Sub Total: x 1048 CGST: 2.5% 3 26.2 SGST: 2.5% 3 26.2 Total: ~ 1100.4 MODE: Cash SAVE PAPER SAVE NATURE !! THANK YOU FOR A DELICIOUS MEAL.'},
 {'filename': 'Meal Invoice 5.pdf',
  'category': 'meal',
  'status': 'Partially Reimbursed',
  'reason': 'Meal amount ≈ ₹400 exceeds ₹200 per‑meal limit.',
  'text': 'WEST HOLLYWOOD 7677 state Los Angeles 1800 000000 Receipt No.: 3574 Table No.: 45 Date: Dec 23, 2024 18:24 QTY/ Item Name Price Amount 2 Coffee 200.00 400.00 Sub Total: 400.00 CGST: 5% 20.00 SGST: 5% 20.00 Total: 440.00 Payment Mode: Online PLEASE VISIT US AGAIN THANK YOU!!'},
 {'filename': 'Meal Invoice 2.pdf',
  'catego